In [ ]:
!pip install -q -U immutabledict sentencepiece
!git clone https://github.com/google/gemma_pytorch.git
!mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/gemma/

In [ ]:
import re
import numpy as np
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

In [ ]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
import sys
sys.path.append("/kaggle/working/gemma_pytorch/") 
from gemma.config import get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM
from gemma.tokenizer import Tokenizer
import contextlib
import torch

In [ ]:
import random
VARIANT = "2b-it" 
weights_dir = '/kaggle/input/gemma/pytorch/2b-it/2' 
device = 'cpu'
ckpt_path = f'{weights_dir}/gemma-{VARIANT}.ckpt'
def seed_everything(seed):
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    torch.manual_seed(seed)
    random.seed(seed)
seed_everything(seed=2024)

In [ ]:
model_config = get_config_for_2b() if "2b" in VARIANT else get_config_for_7b()

model_config.tokenizer = f"{weights_dir}/tokenizer.model"
print(f"model_config:{model_config}")


print(f"model_config.get_dtype():{model_config.get_dtype()}") 
@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
    torch.set_default_dtype(dtype)
    yield
    torch.set_default_dtype(torch.float)
with _set_default_tensor_type(model_config.get_dtype()):
    model = GemmaForCausalLM(model_config)
    print("This is the model structure. : \n", model)
    model.load_weights(ckpt_path)
    print("Model weights loaded.")
    model = model.to(device).eval()
print("Model Configuraiton Done")

In [ ]:
def Ask_model(prompt):
    USER_CHAT_TEMPLATE = '{prompt}'
    MODEL_CHAT_TEMPLATE = '{prompt}'
    prompt = (
        USER_CHAT_TEMPLATE.format(
            prompt=prompt
        )
        + MODEL_CHAT_TEMPLATE.format(prompt="{prompt}")
    )
    output = model.generate(
    USER_CHAT_TEMPLATE.format(prompt=prompt),
    device=device)
    print('done')
    return output 
    
    

In [ ]:
topics = [
    'Write a large document about pollution',
    'Write a large document about Inflation',
    'Write a large document about The unemployment',
    'Write a large document about Overpopulation'
] 

In [ ]:
class preprocessing:
    def __init__(self,text):
        self.text=text
        self.words=None
    def clear(self):
        clean = re.compile('<.*?>')
        self.text =  re.sub(clean, '', self.text)
        self.text= re.sub(r"[^a-zA-Z]"," ",self.text)
        self.text = re.sub(r'\s+', ' ', self.text)
    def Normalization(self):
        self.text=self.text.lower()
        print(self.text)
    def Tokenization(self):
        self.words = word_tokenize(self.text)
        print(self.words)
    def del_stop_words(self):
        stop_words = set(stopwords.words('english'))
        self.words=[word for word in self.words if (word not in stop_words)]
    def stemming(self):
        stemmer = PorterStemmer()
        self.words = [stemmer.stem(word) for word in self.words]
    def unique_words(self):
        return set(self.words)
    def return_string(self):
        return ' '.join(self.words)
    def del_char (self):
        self.words=[word for word in self.words if (len(word)>1)]
        

In [ ]:
def clean_document (document):
    cleaner = preprocessing(document)
    cleaner.clear()
    cleaner.Normalization()
    cleaner.Tokenization()
    cleaner.del_stop_words()
    cleaner.stemming()
    output = cleaner.return_string()
    return output 

In [ ]:
documents = [Ask_model(topic) for topic in topics]

In [ ]:
preprocessed_documents = [clean_document(document) for document in documents]

In [ ]:
preprocessed_documents[3]

In [ ]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(preprocessed_documents)

In [ ]:
feature_names = vectorizer.get_feature_names_out()
tfidf_weights = tfidf_matrix.toarray()

In [ ]:
tfidf_weights

In [ ]:
plt.figure(figsize=(50, 8))  # Adjust figure size as needed
plt.pcolormesh(tfidf_weights, cmap="YlOrRd")


plt.xticks(rotation=90)
plt.xticks(ticks=range(len(feature_names) ), labels=feature_names)


plt.tight_layout()  
plt.show()

In [ ]:
sentences = []
word_set = []

for sent in preprocessed_documents:
    words = [word for word in sent.split()]
    sentences.append(words)
    for word in words:
        if word not in word_set:
            word_set.append(word)
            
word_set = set(word_set)


In [ ]:
word_ind = {}
for i, word in enumerate(word_set):
    word_ind[word] = i

In [ ]:
def count_dict(sentences):
    count_dict = {}
    for word in word_set:
        count_dict[word] = 0
    for sent in sentences:
        for word in sent:
            count_dict[word] += 1
    return count_dict


In [ ]:
word_count = count_dict(sentences)

In [ ]:
def tf(document, word):
    N = len(document)
    occurance = len([token for token in document if token == word])
    return occurance / N

In [ ]:
def idf(word):
    if word in word_count:
        word_occurance = word_count[word] + 1
    else:
        word_occurance = 1
    return 1+ np.log((len(preprocessed_documents) + 1) / word_occurance)

In [ ]:
def tf_idf(sentence):
    vec = np.zeros((len(word_set),))
    for word in sentence:
        tff = tf(sentence, word)
        idff = idf(word)
        vec[word_index[word]] = tff * idff
    return vec

In [ ]:
tfidf_matrix = []
for sent in sentences:
    tfidf_matrix.append(tf_idf(sent))


In [ ]:
tfidf_matrix

In [ ]:
plt.figure(figsize=(50, 8))  # Adjust figure size as needed
plt.pcolormesh(tfidf_matrix, cmap="YlOrRd")


plt.xticks(rotation=90)
plt.xticks(ticks=range(len(word_set) ), labels=word_set)


plt.tight_layout()  
plt.show()